In [ ]:
!unzip 0643.zip -d birbeck

In [253]:
# download libraries

!pip install pytrec-eval-terrier

In [170]:
# import libraries

from nltk.corpus import wordnet as wn
import multiprocessing
import time
import collections
import numpy
from concurrent.futures import ProcessPoolExecutor
import pytrec_eval
from operator import itemgetter
from collections import OrderedDict
import json

In [252]:
# download Wordnet dictionary

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vanam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [169]:
# Splitting the words in Birbeck corpus in pairs of incorrect and correct spellings

f = open("/content/birbeck/ABODAT.643", "r")
# f = open("ota_20.500.12024_0643\\0643\\ABODAT.643")
content = f.readlines()

birbeck_list = []

for line in content:
    if(line.startswith('$')):
        continue
    pairs = line.strip().split(',')
    # print(pairs)
    for words in pairs:
        if(len(words) > 0):
              words = words.strip()
              # print(words.split()[0])
              birbeck_list.append(words)

birbeck_list[110:120]

['sad said',
 'sweety sweaty',
 'dreamd dreamed',
 'stayd stayed',
 'fellt felt',
 'litlle little',
 'nexst next',
 'satt sat',
 'weeckend weekend',
 'catsh catch']

In [4]:
# initializing variables

birbeck_tokens={}
correct_dict = collections.defaultdict(dict)
MED = collections.defaultdict(dict)

In [172]:
# to calculate Levenshtein distance between token 1 and token 2

def levenshtein(token1, token2):
    distances = numpy.zeros((len(token1) + 1, len(token2) + 1))
    for t1 in range(len(token1) + 1): distances[t1][0] = t1
    for t2 in range(len(token2) + 1): distances[0][t2] = t2
    a = 0; b = 0; c = 0

    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            a = distances[t1][t2 - 1]
            b = distances[t1 - 1][t2]
            c = distances[t1 - 1][t2 - 1]

            if (a <= b and a <= c): distances[t1][t2] = a + 1
            elif (b <= a and b <= c): distances[t1][t2] = b + 1
            else: 
                if (token1[t1 - 1] == token2[t2 - 1]): distances[t1][t2] = c
                else: distances[t1][t2] = c + 2


            if(distances[t1][t2] > 10): 
                return

#     print(token1, token2, distances[len(token1)][len(token2)])
    MED[token1].update({token2 : int(distances[len(token1)][len(token2)])})



In [173]:
# Preprocessing Wordnet tokens to check if it is a number or other garbage value  

def comparison(incorrect_token):
    for word in wn.words():
        if(word.startswith('.') or word[0].isdigit()):
            continue
        levenshtein(incorrect_token, word)
    

In [234]:
# words in Wordnet dictionary

count_wn_words = 0
for i in wn.words():
    count_wn_words +=1
print(count_wn_words)


147306


In [22]:
# to create argument list containing all incorrect spelling tokens for parallel processing and 
# to create a dictionary of incorrect word and the corresponding correct word

birbeck_incorrect_tokens=[]

def create_arg_list():
    for idx in range(0,len(birbeck_list)):
        birbeck_incorrect_tokens.append(birbeck_list[idx].split()[0])
        correct_dict[birbeck_list[idx].split()[0]].update({birbeck_list[idx].split()[1] : 1})


In [23]:
# call method

create_arg_list()

In [264]:
list(correct_dict.items())[:5]

[('caugt', {'caught': 1}),
 ('choped', {'chopped': 1}),
 ('senery', {'scenery': 1}),
 ('tak', {'take': 1}),
 ('thougt', {'thought': 1})]

In [265]:
birbeck_incorrect_tokens[:5]

['thougt', "wasen't", 'abut', 'eweryone', 'leeiissy']

In [25]:
for i in birbeck_incorrect_tokens:
    comparison(i)

In [116]:
# multiprocessing pool object
# pool = multiprocessing.Pool()

# pool object with number of element
# pool = multiprocessing.Pool(processes=4)

with ProcessPoolExecutor(max_workers=5) as executor:
        for result in executor.map(comparison, birbeck_incorrect_tokens):
            pass


In [27]:
# write the calculated edit distances to json file

import json

with open('MED1.json', 'w') as fp:
      json.dump(MED, fp, indent=2)

In [3]:
f = open('MED1.json')

MED = json.load(f)

In [ ]:
# list(MED.items())[:1]

In [246]:
# Creating dictionaries to store top k results k = {1, 5, 10} respectively and write to files in system

sort_dict_10 = collections.defaultdict(dict)
sort_dict_5 = collections.defaultdict(dict)
sort_dict_1 = collections.defaultdict(dict)

def sorted_dict_elements():
    for i in MED:
        sort_dict_10[i] =  (OrderedDict(sorted( MED[i].items(), key=itemgetter(1))[:10]))
        sort_dict_5[i] =  (OrderedDict(sorted( MED[i].items(), key=itemgetter(1))[:5]))
        sort_dict_1[i] =  (OrderedDict(sorted( MED[i].items(), key=itemgetter(1))[:1]))
    with open('sort_dict.json', 'w') as fp:
        json.dump(sort_dict_10, fp, indent=2)
        json.dump(sort_dict_5, fp, indent=2)
        json.dump(sort_dict_1, fp, indent=2)

sorted_dict_elements()

In [268]:
list(sort_dict_5.items())[:5]

[('caugt',
  OrderedDict([('cut', 2),
               ('aught', 2),
               ('caput', 2),
               ('cat', 2),
               ('august', 3)])),
 ('choped',
  OrderedDict([('chopped', 1),
               ('choked', 2),
               ('chapped', 3),
               ('co-ed', 3),
               ('cod', 3)])),
 ('senery',
  OrderedDict([('scenery', 1),
               ('eery', 2),
               ('energy', 2),
               ('sentry', 2),
               ('beery', 3)])),
 ('tak',
  OrderedDict([('ak', 1),
               ('tack', 1),
               ('taka', 1),
               ('take', 1),
               ('talk', 1)])),
 ('thougt',
  OrderedDict([('thought', 1),
               ('though', 2),
               ('thou', 2),
               ('thug', 2),
               ('tout', 2)]))]

In [247]:
# Calculating average using pytrec_eval 

results = collections.defaultdict(dict)

for item in correct_dict:
    results[item] = {}
    
    if(list(correct_dict[item].keys())[0] in sort_dict_1[item].keys()):
        results[item][list(correct_dict[item])[0]] = 1
    
    for k in list(sort_dict_5[item].keys()):
        if( k not in results[item].keys()):
            results[item][k] = 1/5
            
    for k in list(sort_dict_10[item].keys()):
        if( k not in results[item].keys()):
            results[item][k] = 1/10    
 

In [248]:
evaluator = pytrec_eval.RelevanceEvaluator(correct_dict, {'success'})
res = evaluator.evaluate(results)


In [ ]:
# print(json.dumps(res, indent=4))

In [250]:
for measure in sorted(list(res[list(res.keys())[0]].keys())):
        print('average', measure, ': ', pytrec_eval.compute_aggregated_measure(measure, [query_measures[measure] for query_measures in eval.values()]))

average success_1 :  0.3577075098814229
average success_10 :  0.5355731225296443
average success_5 :  0.4901185770750988
